In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bengali-news-article-summarization/preprocessed_bangla_news.csv


In [2]:
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
from transformers import MT5ForConditionalGeneration, AutoTokenizer ,DataCollatorForSeq2Seq, Trainer, TrainingArguments
import os
from torch.utils.data import Dataset, DataLoader
from transformers import Trainer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast


In [3]:
df = pd.read_csv('/kaggle/input/bengali-news-article-summarization/preprocessed_bangla_news.csv', usecols=['summary', 'article'])

In [4]:
df.dropna()

,article,summary
0,তুরস্কের পশ্চিমাঞ্চলীয় ইজমির প্রদেশে সুইডেনের ...,তুরস্কের পশ্চিমাঞ্চলীয় ইজমির প্রদেশে সুইডেনের ...
1,সারাদেশে ডেঙ্গু পরিস্থিতি দিন দিন আরও ভয়াবহ রূ...,সারাদেশে ডেঙ্গু পরিস্থিতি দিন দিন আরও ভয়াবহ রূ...
2,শোকাবহ আগস্টের প্রথম দিনে সুনামগঞ্জে স্বেচ্ছায়...,শোকাবহ আগস্টের প্রথম দিনে সুনামগঞ্জে স্বেচ্ছায়...
3,পানিসম্পদ উপমন্ত্রী একেএম এনামুল হক শামীম বলেছ...,পানিসম্পদ উপমন্ত্রী একেএম এনামুল হক শামীম বলেছ...
4,বিএনপির ‌‘অগ্নিসন্ত্রাস ও নৈরাজ্য সৃষ্টির প্রত...,বিএনপির ‌‘অগ্নিসন্ত্রাস ও নৈরাজ্য সৃষ্টির প্রত...
...,...,...
39136,"আমার ভাই মনে কষ্ট নিয়ে চলে গেছে। সে মনে করত, ত...","আমার ভাই মনে কষ্ট নিয়ে চলে গেছে। সে মনে করত, ত..."
39137,সারাজীবন আমি ভেবেছি সাদি আর শিবলী আমরা প্রায় স...,সারাজীবন আমি ভেবেছি সাদি আর শিবলী আমরা প্রায় স...
39138,দেশে শিশুশ্রম বেড়েছে। বাংলাদেশ পরিসংখ্যান ব্যু...,দেশে শিশুশ্রম বেড়েছে। বাংলাদেশ পরিসংখ্যান ব্যু...
39139,‘সেবার ব্রতে চাকরি’ এই শ্লোগানে শতভাগ মেধা-যোগ...,‘সেবার ব্রতে চাকরি’ এই শ্লোগানে শতভাগ মেধা-যোগ...


In [5]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.30, shuffle=True)


In [6]:
from datasets import Dataset
ds_train = Dataset.from_pandas(df_train)
ds_test = Dataset.from_pandas(df_test)

In [7]:
ds_train['summary'][50]

'একটি দেশের অর্থনীতিতে ব্যাংকিং সেক্টর অত্যন্ত গুরুত্বপূর্ণ ভূমিকা পালন করে থাকে। বিশেষ করে বাংলাদেশের মতো একটি উন্নয়নশীল দেশের জন্য ব্যাংকিং সেক্টরের ভূমিকা অত্যন্ত তাৎপর্যপূর্ণ। উন্নয়নশীল দেশগুলোর ব্যক্তি খাতের উদ্যোক্তাগণ সব সময়ই পুঁজির স্বল্পতায় ভোগেন। উদ্যোক্তাদের পুঁজি বা অর্থায়ন সমস্যা সমাধানে ব্যাংকিং সেক্টর সহায়ক ভূমিকা পালন করে। উন্নত দেশগুলোতে দীর্ঘমেয়াদি পুঁজির জোগান নিশ্চিত করার জন্য সাধারণত স্টক মার্কেটের ওপর নির্ভর করা হয়। কিন্তু বাংলাদেশের স্টক মার্কেট এখনো সেভাবে বিকশিত হতে পারেনি। ফলে ব্যক্তি খাতের উদ্যোক্তাদের পুঁজি চাহিদা মেটানোর জন্য সবচেয়ে নির্ভরযোগ্য অবলম্বন হচ্ছে ব্যাংক। কিন্তু এটা কোনোভাবেই অস্বীকার করা যাবে না যে আমাদের দেশের ব্যাংকিং ব্যবস্থা নানা সমস্যায় জর্জরিত হয়ে পড়েছে। ফলে ব্যাংকিং সেক্টর থেকে উদ্যোক্তাগণ প্রত্যাশিত মাত্রায় সাপোর্ট পাচ্ছেন না।'

In [8]:
tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBART")


tokenizer_config.json:   0%|          | 0.00/498 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/832 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/221 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
def tokenize_data(data):
    input_features = tokenizer(data['article'], truncation=True, max_length=1024)
    label = tokenizer(data['summary'],truncation =True, max_length=128)
    return {
        'input_ids':input_features['input_ids'],
        'attention_mask': input_features['attention_mask'],
        'labels': label['input_ids'],
    }


In [10]:
tokenized_ds_train = ds_train.map(
    tokenize_data,
    batched=True,
    remove_columns=ds_train.column_names  # Remove original columns
)

# Apply tokenization to the test dataset
tokenized_ds_test = ds_test.map(
    tokenize_data,
    batched=True,
    remove_columns=ds_test.column_names
)

Map:   0%|          | 0/27398 [00:00<?, ? examples/s]

Map:   0%|          | 0/11743 [00:00<?, ? examples/s]

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/IndicBART")

pytorch_model.bin:   0%|          | 0.00/976M [00:00<?, ?B/s]

In [12]:
data_collator = DataCollatorForSeq2Seq(
    tokenizer = tokenizer,
    model = model,
    return_tensors = 'pt'
)

In [13]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    eval_strategy='epoch',
    learning_rate=5e-4,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps = 16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    logging_dir='./logs',
    logging_steps=10,
    generation_max_length = 128,
    push_to_hub = False
)

In [14]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds_train,
    eval_dataset=tokenized_ds_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [15]:
# Disable WandB if not needed
import os
os.environ["WANDB_DISABLED"] = "true"

In [16]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
0,0.015000,0.008313


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/parallel_apply.py:79: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.device(device), torch.cuda.stream(stream), autocast(enabled=autocast_enabled):
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


TrainOutput(global_step=428, training_loss=0.17701744128053434, metrics={'train_runtime': 5480.6555, 'train_samples_per_second': 4.999, 'train_steps_per_second': 0.078, 'total_flos': 2.967278442263347e+16, 'train_loss': 0.17701744128053434, 'epoch': 0.9997080291970802})

In [18]:
pip install rouge_score

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=f81f09da1f0b7f09b01fe0b1cfefade858f302e95ce72f66b2b3d7703b3ca53c
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


# Evaluate ROUGE scores

In [20]:
import evaluate
import numpy as np
from nltk.tokenize import RegexpTokenizer

rouge_metric = evaluate.load("rouge")

# define function for custom tokenization
def tokenize_sentence(arg):
    encoded_arg = tokenizer(arg)
    return tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

# define function to get ROUGE scores with custom tokenization
def metrics_func(eval_arg):
    preds, labels = eval_arg

    # Replace -100
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Convert id tokens to text

    text_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    text_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Insert a line break (\n) in each sentence for ROUGE scoring

    # (Note : Please change this code, when you perform on other languages except for Bengali)
    text_preds = [(p if p.endswith(("!", "!", "?", "?", "।")) else p + "।") for p in text_preds]
    text_labels = [(l if l.endswith(("!", "!", "?", "?", "।")) else l + "।") for l in text_labels]
    sent_tokenizer_bn = RegexpTokenizer(u'[^!!??।]*[!!??।]')
    text_preds = ["\n".join(np.char.strip(sent_tokenizer_bn.tokenize(p))) for p in text_preds]
    text_labels = ["\n".join(np.char.strip(sent_tokenizer_bn.tokenize(l))) for l in text_labels]

    # compute ROUGE score with custom tokenization
    return rouge_metric.compute(
    predictions=text_preds,
    references=text_labels,
    tokenizer=tokenize_sentence
    )

In [21]:
from torch.utils.data import DataLoader

sample_dataloader = DataLoader(
    tokenized_ds_test.with_format("torch"),
    collate_fn=data_collator,
    batch_size=5
)

for batch in sample_dataloader:
    with torch.no_grad():
        preds = model.generate(
            batch["input_ids"].to(device),
            num_beams=15,
            num_return_sequences=1,
            no_repeat_ngram_size=1,
            remove_invalid_values=True,
            max_length=128
        )
    labels = batch["labels"]
    break

metrics_func([preds, labels])

/opt/conda/lib/python3.10/site-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


{'rouge1': 0.31972322255625996,
 'rouge2': 0.11360965095904856,
 'rougeL': 0.20695271922212544,
 'rougeLsum': 0.22878913063662437}